In [1]:
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Считываем данные

In [2]:
column_names = ['user_id', 'item_id', 'rating','timestamp']
df = pd.read_csv('/Users/mirpulatov/Documents/3s-RecommenderSystems/ml-100k/u.data', sep='\t', names=column_names)

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
names= ['item_id', 'movie title', 'release date', 'video release date',
              'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
              'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
              'Thriller', 'War', 'Western' ]
df_items = pd.read_csv("/Users/mirpulatov/Documents/3s-RecommenderSystems/ml-100k/u.item", names= names, sep='|', encoding = 'ISO-8859-1')

In [5]:
df_items.head()

,item_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Количество комбинаций user_id и item_id

In [6]:
np.unique(df_items.item_id).shape[0] * np.unique(df.user_id).shape[0]

1586126

# Обработаем Items

Пропущенные значения

In [7]:
from sklearn.impute import MissingIndicator

In [8]:
indicator = MissingIndicator()
missing_values_items = indicator.fit_transform(df_items)
print(np.sum(missing_values_items))

1686


In [9]:
df_items[missing_values_items].head()

,item_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Все пропуски в столбце video release date, можем удалить этот столбец и еще столбец IMDb URL

In [10]:
df_items.drop(['video release date', 'IMDb URL'],  axis=1, inplace=True)

In [11]:
df_items.head()

,item_id,movie title,release date,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


Проверим еще раз

In [12]:
missing_values_items = indicator.fit_transform(df_items)
print(np.sum(missing_values_items))

1


Осталось одно значение, удалим его

In [13]:
df_items.drop(266, axis=0, inplace=True)

Можем удалить дату из столбца title, также преобразуем его к lowercase и удалим ненужные символы

In [14]:
def string_preprocess(row):
    return " ".join(re.findall("[a-zA-Z]+", row)).lower()

In [15]:
df_items['movie title'] = df_items['movie title'].apply(string_preprocess)

In [16]:
df_items.head()

,item_id,movie title,release date,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,toy story,01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,goldeneye,01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,four rooms,01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,get shorty,01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,copycat,01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


Преобразуем дату, достаточно оставить только год

In [17]:
df_items['release date'] = pd.to_datetime(df_items['release date'])

In [18]:
df_items['release date'] = df_items['release date'].dt.year

In [19]:
df_items.head()

,item_id,movie title,release date,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,toy story,1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,goldeneye,1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,four rooms,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,get shorty,1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,copycat,1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


Преобразуем название фильма в вектор

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_items['movie title'])

In [22]:
tfidf_matrix.shape

(1681, 2337)

In [23]:
df_tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), index=df_items['movie title'].values)

In [24]:
df_tfidf_matrix.reset_index(inplace=True)

In [25]:
df_tfidf_matrix.rename(columns={'index': 'movie title'}, inplace=True)

In [26]:
pivot_items = pd.merge(
    df_items,
    df_tfidf_matrix,
    on=['movie title']
)

In [27]:
pivot_items.drop('movie title', inplace=True, axis=1)

In [28]:
pivot_items.head()

,item_id,release date,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336
0,1,1995,0,0,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1995,0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1995,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1995,0,1,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1995,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Обработаем Users

In [29]:
df.drop('timestamp', axis=1, inplace=True)

In [30]:
df.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


# Генерируем результирующие таблицы

Все комбинации пользователей и фильмов

In [31]:
np.unique(df_items.item_id).shape[0] * np.unique(df.user_id).shape[0]

1585183

In [32]:
data = np.array(np.meshgrid(np.unique(df.user_id), np.unique(df_items.item_id))).T.reshape(-1, 2)
pivot_df = pd.DataFrame(data, columns=['user_id', 'item_id'])

In [33]:
df_tmp = df.drop('rating', axis=1)

In [34]:
test = pd.concat([pivot_df, df_tmp]).drop_duplicates(keep=False)

In [35]:
df_items.head()

,item_id,movie title,release date,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,toy story,1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,goldeneye,1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,four rooms,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,get shorty,1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,copycat,1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [36]:
train_df = pd.merge(
    df,
    pivot_items,
    how='inner',
    on='item_id'
)

In [37]:
# train_df.to_csv('train.csv', index=False)

In [38]:
train_df.head()

,user_id,item_id,rating,release date,unknown,Action,Adventure,Animation,Children,Comedy,...,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336
0,196,242,3,1997,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,63,242,3,1997,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,226,242,5,1997,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,154,242,3,1997,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,306,242,5,1997,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
test.shape

(1485201, 2)

In [45]:
test_1 = test[:100000].copy() # 1
test_2 = test[100000:200000].copy() # 2
test_3 = test[200000:300000].copy() # 3
test_4 = test[300000:400000].copy() # 4
test_5 = test[400000:500000].copy() # 5
test_6 = test[500000:600000].copy() # 6
test_7 = test[600000:700000].copy() # 7
test_8 = test[700000:800000].copy() # 8
test_9 = test[800000:900000].copy() # 9
test_10 = test[900000:1000000].copy() # 10
test_11 = test[1000000:1100000].copy() # 11
test_12 = test[1100000:1200000].copy() # 12
test_13 = test[1200000:1300000].copy() # 13
test_14 = test[1300000:1400000].copy() # 14
test_15 = test[1400000:].copy() # 15

In [47]:
test_df1 = pd.merge(
    test_1,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df1.to_csv('test_1.csv', index=False)

In [48]:
test_df2 = pd.merge(
    test_2,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df2.to_csv('test_2.csv', index=False)

In [49]:
test_df3 = pd.merge(
    test_3,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df3.to_csv('test_3.csv', index=False)

In [50]:
test_df4 = pd.merge(
    test_4,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df4.to_csv('test_4.csv', index=False)

In [52]:
test_df5 = pd.merge(
    test_5,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df5.to_csv('test_5.csv', index=False)

In [53]:
test_df6 = pd.merge(
    test_6,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df1.to_csv('test_6.csv', index=False)

In [54]:
test_df7 = pd.merge(
    test_7,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df7.to_csv('test_7.csv', index=False)

In [55]:
test_df8 = pd.merge(
    test_8,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df8.to_csv('test_8.csv', index=False)

In [56]:
test_df9 = pd.merge(
    test_9,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df9.to_csv('test_9.csv', index=False)

In [57]:
test_df10 = pd.merge(
    test_10,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df10.to_csv('test_10.csv', index=False)

In [58]:
test_df11 = pd.merge(
    test_11,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df11.to_csv('test_11.csv', index=False)

In [59]:
test_df12 = pd.merge(
    test_12,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df12.to_csv('test_12.csv', index=False)

In [60]:
test_df13 = pd.merge(
    test_13,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df13.to_csv('test_13.csv', index=False)

In [61]:
test_df14 = pd.merge(
    test_14,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df14.to_csv('test_14.csv', index=False)

In [62]:
test_df15 = pd.merge(
    test_15,
    pivot_items,
    how='inner',
    on='item_id'
)
test_df15.to_csv('test_15.csv', index=False)